In [1]:
try:
    from google.colab import drive
    %pip install git+https://github.com/denis-spe/exttorch.git
# !pip install torch~=2.1.0 torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html
except ImportError:
    import sys
    sys.path.append('/Volumes/Storage/DS/DL/exttorch/src/')

**Iris**

In [2]:
import torch
from torch import nn
from exttorch.optimizers import Adam, SGD
from exttorch.losses import CrossEntropyLoss, MSELoss
from sklearn.datasets import load_iris, load_wine
from exttorch.models import Sequential
from exttorch.metrics import Precision, Recall

In [3]:
x, y = load_iris(return_X_y=True)
print(x.shape, y.shape)

(150, 4) (150,)


In [4]:
set(y)

{0, 1, 2}

In [5]:
iris_model = Sequential([
    nn.Linear(4, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 3)
])

iris_model.compile(
    optimizer=Adam(),
    loss=CrossEntropyLoss(),
    metrics=['accuracy']
            # Precision(average="weighted"),
            # Recall(average="weighted")]
    )

history = iris_model.fit(x, y, epochs=10, validation_split=0.30)


Epoch 1/10
105/105 ✔️ ━━━━━━━━━━━━━━━━━━━━ 15s 295ms/step  - loss: 1.0634 - accuracy: 0.3335 - val_loss: 0.9272 - val_accuracy: 0.2776
Epoch 2/10
105/105 ✔️ ━━━━━━━━━━━━━━━━━━━━ 3s 456ms/step  - loss: 0.8450 - accuracy: 0.3421 - val_loss: 0.7325 - val_accuracy: 0.2776
Epoch 3/10
105/105 ✔️ ━━━━━━━━━━━━━━━━━━━━ 4s 513ms/step  - loss: 0.6207 - accuracy: 0.3421 - val_loss: 0.5982 - val_accuracy: 0.2865
Epoch 4/10
105/105 ✔️ ━━━━━━━━━━━━━━━━━━━━ 6s 963ms/step  - loss: 0.4888 - accuracy: 0.3406 - val_loss: 0.5006 - val_accuracy: 0.3797
Epoch 5/10
105/105 ✔️ ━━━━━━━━━━━━━━━━━━━━ 3s 306ms/step  - loss: 0.4145 - accuracy: 0.3588 - val_loss: 0.4220 - val_accuracy: 0.3909
Epoch 6/10
105/105 ✔️ ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step  - loss: 0.3621 - accuracy: 0.3582 - val_loss: 0.3606 - val_accuracy: 0.3952
Epoch 7/10
105/105 ✔️ ━━━━━━━━━━━━━━━━━━━━ 1s 669ms/step  - loss: 0.3196 - accuracy: 0.3580 - val_loss: 0.3098 - val_accuracy: 0.3958
Epoch 8/10
105/105 ✔️ ━━━━━━━━━━━━━━━━━━━━ 3s 188ms/step  - 

In [ ]:
history.history

{'accuracy': [0.3335,
  0.3421,
  0.3421,
  0.3406,
  0.3588,
  0.3582,
  0.358,
  0.358,
  0.3573,
  0.3572],
 'val_accuracy': [0.2776,
  0.2776,
  0.2865,
  0.3797,
  0.3909,
  0.3952,
  0.3958,
  0.3958,
  0.3958,
  0.3958],
 'loss': [1.0634,
  0.845,
  0.6207,
  0.4888,
  0.4145,
  0.3621,
  0.3196,
  0.2828,
  0.2521,
  0.2254],
 'val_loss': [0.9272,
  0.7325,
  0.5982,
  0.5006,
  0.422,
  0.3606,
  0.3098,
  0.2668,
  0.2317,
  0.2029]}

## **Wine Dataset**

In [7]:
w_x, w_y = load_wine(return_X_y=True)
w_x.shape, w_y.shape

((178, 13), (178,))

In [8]:
set(w_y)

{0, 1, 2}

In [9]:
wine_model = Sequential([
    nn.Linear(13, 512),
    nn.ReLU(),
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Linear(512, 3)
])

wine_model.compile(
    optimizer=Adam(),
    loss=CrossEntropyLoss(),
    metrics=['accuracy']
    )

wine_model.fit(w_x, w_y, epochs=10, validation_split=0.25)


Epoch 1/10
134/134 ✔️ ━━━━━━━━━━━━━━━━━━━━ 14s 200ms/step  - loss: 34.3560 - accuracy: 0.3553 - val_loss: 3.2153 - val_accuracy: 0.1971
Epoch 2/10
134/134 ✔️ ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step  - loss: 1.9630 - accuracy: 0.3667 - val_loss: 0.6352 - val_accuracy: 0.3543
Epoch 3/10
134/134 ✔️ ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step  - loss: 0.9942 - accuracy: 0.3583 - val_loss: 0.6102 - val_accuracy: 0.5170
Epoch 4/10
134/134 ✔️ ━━━━━━━━━━━━━━━━━━━━ 3s 216ms/step  - loss: 0.8130 - accuracy: 0.3391 - val_loss: 0.8717 - val_accuracy: 0.4458
Epoch 5/10
134/134 ✔️ ━━━━━━━━━━━━━━━━━━━━ 3s 213ms/step  - loss: 0.7394 - accuracy: 0.3537 - val_loss: 0.7870 - val_accuracy: 0.4458
Epoch 6/10
134/134 ✔️ ━━━━━━━━━━━━━━━━━━━━ 3s 215ms/step  - loss: 0.7461 - accuracy: 0.3501 - val_loss: 0.6955 - val_accuracy: 0.5119
Epoch 7/10
134/134 ✔️ ━━━━━━━━━━━━━━━━━━━━ 5s 209ms/step  - loss: 0.7531 - accuracy: 0.3523 - val_loss: 0.7101 - val_accuracy: 0.4489
Epoch 8/10
134/134 ✔️ ━━━━━━━━━━━━━━━━━━━━ 6s 214ms/step  -

In [16]:
wine_model.evaluate(w_x, w_y)

178/178 ✔️ ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step  - val_loss: 0.7415 - val_accuracy: 0.5653


{'val_loss': 0.7415, 'val_accuracy': 0.5653}

In [17]:
wine_model.predict(w_x[:5])

ImportError: cannot import name 'SinglePredictionsFormat' from 'exttorch._data_handle' (/Volumes/Storage/DS/DL/exttorch/src/exttorch/_data_handle.py)